## classification 

In [1]:
import numpy as np
import os
import cv2
import keras
import glob
from keras.utils import np_utils
from keras.layers import Input, Dense, concatenate, Flatten, Dropout, multiply, BatchNormalization, Activation, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, load_model
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.initializers import TruncatedNormal, he_normal
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from random import shuffle

/Users/vyettella/anaconda3/envs/insight/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class_folders = os.listdir('./classification_training_data/')
class_folders.remove('.DS_Store')
print(class_folders)

['date_palm', 'oak', 'weeping_wattle', 'white_mulberry', 'carob', 'golden_rain_tree', 'common_myrtle', 'poplar', 'privet', 'laurel', 'black_locust', 'pine', 'oriental_plane', 'eucalyptus', 'persian_lilac', 'judas', 'tree_of_heaven', 'mediterranean_cypress', 'hibiscus', 'olive', 'silky_oak', 'weeping_willow', 'japanese_pagoda']


In [39]:
files = []
for class_folder in class_folders:
    files.append(glob.glob('./classification_training_data/' + class_folder + '/*.jpg'))

In [5]:
dataset = []
for class_num, file in enumerate(files):
    for fname in file:
        dataset.append({"x": fname, "y": class_num})

In [41]:
class_folders

['date_palm',
 'oak',
 'weeping_wattle',
 'white_mulberry',
 'carob',
 'golden_rain_tree',
 'common_myrtle',
 'poplar',
 'privet',
 'laurel',
 'black_locust',
 'pine',
 'oriental_plane',
 'eucalyptus',
 'persian_lilac',
 'judas',
 'tree_of_heaven',
 'mediterranean_cypress',
 'hibiscus',
 'olive',
 'silky_oak',
 'weeping_willow',
 'japanese_pagoda']

In [42]:
dataset

[{'x': './classification_training_data/date_palm/tree-1.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-134.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-120.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-108.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-81.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-95.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-42.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-56.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-57.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-43.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-94.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-80.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-109.jpg', 'y': 0},
 {'x': './classification_training_data/date_palm/tree-121.jpg

In [6]:
class_

22

In [7]:
indexes = [i for i in range(len(dataset))]
shuffle(indexes)
split_idx = int(len(indexes) * 0.8)
train_idx = indexes[0:split_idx]
val_idx = indexes[split_idx::]
print("Size Training set: {} | Valset {}".format(len(train_idx), len(val_idx)))


Size Training set: 2308 | Valset 578


In [8]:
def generator_xy(batch_sz, dataset, index_set, is_training):
    idx = 0 # 'pointer for indexes
    while 1:
        if is_training:
            shuffle(index_set)
        x_batch = np.zeros( (batch_sz, 224, 224, 3), dtype=np.float32)
        y_batch = np.zeros( (batch_sz, 23), dtype=np.float32)
        for row in range(batch_sz):
            example = dataset[indexes[idx]]
            
            img_p = example["x"]
            y = example["y"]
            
            bgr = cv2.imread(img_p)
            bgr = cv2.resize(bgr, (224, 224))
            rgb = bgr[:,:,::-1]/255.
            x_batch[row] = rgb[:,:,:]
            y_batch[row, y] = 1
            idx += 1
            if (idx > len(indexes)-1):
                shuffle(indexes)
                idx = 0

        yield x_batch, y_batch

In [43]:
cv2.resize?

In [38]:
batch_sz = 15
steps_per_epoch = len(train_idx) // batch_sz
validation_steps = len(val_idx) // batch_sz


base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for i, layer in enumerate(base_model.layers):
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
output = Dense(23, activation="softmax", name="y")(x) #predict x coordinates of vertices
model = Model(inputs=base_model.input, outputs=output)
adam = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = adam)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 111, 111, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 111, 111, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [10]:
n_epochs = 20
history = model.fit_generator(generator=generator_xy(batch_sz, dataset, train_idx, is_training=1),\
                              validation_data=generator_xy(batch_sz, dataset, val_idx, is_training=0),\
                              steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=n_epochs,\
                              verbose=True)

1967
Epoch 1/10
2754
2147
1449
871
86
2074
1977
1216
1869
2687
358
714
182
2374
722
1074
1480
2688
1538
144
777
2860
1799
1632
2786
2572
2380
2649
674
2780
562
2046
627
920
2557
1705
790
2740
2650
536
751
881
1255
1575
1355
386
1471
320
1091
134
2106
850
1330
744
583
2872
1147
19
1672
1770
2224
1811
171
557
2168
2246
2644
1389
2615
1390
598
1273
2081
138
1937
2857
2149
1057
2273
1541
297
444
1905
821
2111
285
305
2069
1167
310
2256
389
1393
2523
2456
31
2652
1910
1537
2661
2868
1593
1969
2197
235
2260
1880
1680
2826
343
1873
270
2438
1203
417
2251
2756
1258
865
1886
2345
281
2448
2614
929
42
2176
2852
1225
1832
245
1303
1044
2241
2058
1747
780
1373
1915
873
634
2127
1897
561
917
210
326
2232
2469
1534
439
314
1081
1757
465
1263
784
976
453
772
340
529
932
1760
  1/153 [..............................] - ETA: 8:46 - loss: 3.4801 - acc: 0.0000e+00643
1640
1415
644
1807
2495
2302
430
1055
248
909
2320
1342
102
2855
  2/153 [..............................] - ETA: 6:07 - loss: 3.4710 - acc: 

error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize


In [14]:
img_p

NameError: name 'img_p' is not defined

In [24]:
bgr = cv2.imread('./classification_training_data/black_locust/tree-90.jpg')

In [25]:
bgr = cv2.resize(bgr, (224, 224))

error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize


In [30]:
tree_species = os.listdir('./classification_training_data')
tree_species   
tree_species.remove('.DS_Store')

In [36]:
for species in tree_species:
    files = glob.glob('./classification_training_data/' + species + '/*.jpg')
    for file in files:
        try:
            bgr = cv2.imread(file)
            bgr = cv2.resize(bgr, (224, 224))
        except:
            print(file)


./classification_training_data/black_locust/tree-90.jpg


In [ ]:
['black_locust',
 'golden_rain_tree',
 'mediterranean_cypress',
 'oak',
 'persian_lilac',
 'poplar',
 'date_palm',
 'tree_of_heaven',
 'weeping_willow',
 'eucalyptus',
 'pine',
 'privet',
 'hibiscus',
 'judas',
 'japanese_pagoda',
 'weeping_wattle',
 'olive',
 'silky_oak',
 'common_myrtle',
 'oriental_plane',
 'carob',
 'white_mulberry',
 'tree_classifier_inceptionv3',
 'laurel']